# Power plants `14 points`

Source: https://datasets.wri.org/dataset/globalpowerplantdatabase

Description: A comprehensive, global, open source database of power plants

**Topics:**

* Filtering
* Aggregation
* Making maps

In [28]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", '{:,.2f}'.format)

## Basic analysis `2 points`

In [29]:
df = pd.read_csv('global_power_plant_database.csv')

df

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,generation_data_source,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.00,32.32,65.12,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009793,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.00,31.67,65.80,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.00,31.62,65.79,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.00,34.56,69.48,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009795,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.00,34.64,69.72,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009797,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,WRI1022386,50.00,-12.97,28.63,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1089529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.79,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
34932,ZMB,Zambia,Nkana,WRI1022384,20.00,-12.82,28.20,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1043097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.51,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
34933,ZMB,Zambia,Victoria Falls,WRI1022380,108.00,-17.92,25.85,Hydro,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1033763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,575.78,575.78,548.94,579.90,578.32,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,GEODB0040404,920.00,-18.38,26.47,Coal,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1033856,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,785.10",NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1


### According to this dataset, what ten countries have the most power plants?

In [30]:
df['country_long'].value_counts().head(10)

United States of America    9833
China                       4235
United Kingdom              2751
Brazil                      2360
France                      2155
India                       1589
Germany                     1309
Canada                      1159
Spain                        829
Russia                       545
Name: country_long, dtype: int64

### According to this dataset, what ten countries are producing (or can produce) the most energy?

In [31]:
df.groupby('country_long').sum()['capacity_mw'].sort_values(ascending=False)

country_long
China                      1,415,067.38
United States of America   1,204,638.05
India                        316,088.55
Russia                       228,220.05
Japan                        215,365.85
                               ...     
Guinea-Bissau                     18.24
Antarctica                         7.60
Palestine                          7.60
Suriname                           5.00
Saint Lucia                        3.00
Name: capacity_mw, Length: 167, dtype: float64

### What is the most common kind of power plant?

In [32]:
finalranking = []
columns = ['primary_fuel','other_fuel1','other_fuel2','other_fuel3']

for pwtype in columns:
    rank = pd.DataFrame(df[pwtype].value_counts())
    rank.columns = ['total']
    finalranking.append(rank)

pd.concat(finalranking).reset_index().groupby('index').sum('total').sort_values('total',ascending=False)

,total
index,
Solar,10718
Hydro,7191
Wind,5358
Gas,4512
Oil,3568
Coal,2420
Biomass,1506
Waste,1182
Nuclear,195


### What kinds of power plants produce the most energy, in total?

In [33]:
# Considering only primary_fuel here

worldenergy = df.groupby('primary_fuel').sum()['capacity_mw'].sort_values(ascending=False)

pd.DataFrame(worldenergy)

,capacity_mw
primary_fuel,
Coal,"1,965,541.00"
Gas,"1,493,050.60"
Hydro,"1,053,159.62"
Nuclear,"407,911.76"
Wind,"263,053.73"
Oil,"261,878.71"
Solar,"188,312.32"
Biomass,"34,281.30"
Waste,"14,748.71"


## Green energy `2 points`

### Create a new column for each power plant, `is_green`

Include... whatever you feel is actually green. Maybe Hydro, nuclear, wind, solar, geothermal, wave/tidal. `0` means it isn't green, `1` means it is green.

In [34]:
green_dict = {
    'Wind': 1,
    'Solar': 1,
    'Hydro': 1,
    'Geothermal': 1,
    'Wave and Tidal': 1,
}
df['is_green'] = df.primary_fuel.replace(green_dict)

# 
# df['greek'] = (df['cuisine'] == 'Greek').astype(int)

# Same thing with the is_green column for the powerplants one, except you can use .isin instead of a simple == comparison.

In [35]:
df

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,generation_data_source,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017,is_green
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.00,32.32,65.12,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009793,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.00,31.67,65.80,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,1
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.00,31.62,65.79,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,1
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.00,34.56,69.48,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009795,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.00,34.64,69.72,Hydro,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1009797,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,WRI1022386,50.00,-12.97,28.63,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1089529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.79,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,Oil
34932,ZMB,Zambia,Nkana,WRI1022384,20.00,-12.82,28.20,Oil,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1043097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.51,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,Oil
34933,ZMB,Zambia,Victoria Falls,WRI1022380,108.00,-17.92,25.85,Hydro,NaN,NaN,NaN,NaN,ZESCO,Energy Regulation Board of Zambia,http://www.erb.org.zm/reports/EnergySectorRepo...,Power Africa,1033763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,575.78,575.78,548.94,579.90,578.32,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,1
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,GEODB0040404,920.00,-18.38,26.47,Coal,NaN,NaN,NaN,NaN,NaN,GEODB,http://globalenergyobservatory.org,GEODB,1033856,"2,017.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,785.10",NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,Coal


### What percent of the world's energy is green?

In [36]:
# Considering only primary_fuel here

df_wenergy = pd.DataFrame(worldenergy).reset_index()

green_dict = {
    'Wind': 1,
    'Solar': 1,
    'Hydro': 1,
    'Geothermal': 1,
    'Wave and Tidal': 1,
}

df_wenergy['is_green'] = df_wenergy.primary_fuel.replace(green_dict)

ranking_capacity = df_wenergy.groupby('is_green').sum().sort_values('capacity_mw',ascending=False)

ranking_capacity

,capacity_mw
is_green,
Coal,"1,965,541.00"
1,"1,517,765.62"
Gas,"1,493,050.60"
Nuclear,"407,911.76"
Oil,"261,878.71"
Biomass,"34,281.30"
Waste,"14,748.71"
Cogeneration,"4,048.00"
Other,"3,612.86"


In [37]:
# ~ 26% of the world's energy is considered "green"
ranking_capacity['capacity_percent'] = ranking_capacity['capacity_mw']/ranking_capacity['capacity_mw'].sum()*100

ranking_capacity

,capacity_mw,capacity_percent
is_green,,
Coal,"1,965,541.00",34.44
1,"1,517,765.62",26.59
Gas,"1,493,050.60",26.16
Nuclear,"407,911.76",7.15
Oil,"261,878.71",4.59
Biomass,"34,281.30",0.60
Waste,"14,748.71",0.26
Cogeneration,"4,048.00",0.07
Other,"3,612.86",0.06


In [38]:
# ~ 67% of the *power plants* are considered "green"
# df['is_green'].value_counts(normalize=True)

### What countries use the highest percentage of green energy? `2 points`

This one will probably be tricky, and might take a few steps.

In [39]:
# Highest percentage of green energy in the world
greenrank = pd.DataFrame(df[df['is_green'] == 1].groupby('country_long').sum()['capacity_mw'])
greenrank['green_percent_world'] = greenrank['capacity_mw']/greenrank['capacity_mw'].sum()*100

greenrank.sort_values('green_percent_world',ascending=False).head(10)

,capacity_mw,green_percent_world
country_long,,
China,"364,843.88",24.04
United States of America,"247,993.35",16.34
Brazil,"110,141.71",7.26
Canada,"94,663.40",6.24
India,"74,757.72",4.93
Russia,"46,548.44",3.07
United Kingdom,"38,280.58",2.52
Japan,"33,811.90",2.23
France,"33,723.33",2.22


In [41]:
# Highest percentage of green energy in each country
rank_country = df.groupby('country_long').sum()['capacity_mw']

greenrank = pd.merge(rank_country, greenrank, how = 'inner', on = 'country_long', suffixes=('_total', '_green'))


In [43]:
greenrank['green_percent_country'] = greenrank['capacity_mw_green']/greenrank['capacity_mw_total']*100

greenrank.sort_values('green_percent_country',ascending=False).head(10)

,capacity_mw_total,capacity_mw_green,green_percent_world,green_percent_country
country_long,,,,
Lesotho,72.00,72.00,0.00,100.00
Burundi,57.17,57.17,0.00,100.00
Ethiopia,"4,023.90","4,023.90",0.27,100.00
Iceland,"2,484.60","2,484.60",0.16,100.00
Bhutan,"1,482.20","1,482.20",0.10,100.00
Mali,311.50,311.50,0.02,100.00
Mozambique,"2,285.00","2,285.00",0.15,100.00
Nepal,587.95,587.95,0.04,100.00
Palestine,7.60,7.60,0.00,100.00


### Do you feel like you might need to add some more requirements to be considered a country with 100% green energy? If so, filter and re-calculate.

In [24]:
# Removing hydro 

really_green_dict = {
    'Wind': 1,
    'Solar': 1,
    'Geothermal': 1,
    'Wave and Tidal': 1,
}

df['is_really_green'] = df.primary_fuel.replace(really_green_dict)

In [25]:
rk_reallygreen = pd.DataFrame(df.groupby(['country_long','is_really_green']).sum()['capacity_mw']).reset_index()
rk_reallygreen = rk_reallygreen[rk_reallygreen['is_really_green'] == 1]

rk_reallygreen

,country_long,is_really_green,capacity_mw
0,Afghanistan,1,20.00
5,Algeria,1,670.80
11,Antarctica,1,1.00
13,Argentina,1,727.38
23,Australia,1,"9,898.30"
...,...,...,...
588,United States of America,1,"146,336.45"
600,Uruguay,1,"1,606.96"
605,Uzbekistan,1,100.00
612,Vietnam,1,"1,804.40"


In [46]:
rk_reallygreen = pd.merge(rank_country, rk_reallygreen, how = 'inner', on = 'country_long', suffixes=('_total', '_really_green'))

rk_reallygreen['really_green_percent_country'] = rk_reallygreen['capacity_mw_really_green']/rk_reallygreen['capacity_mw_total']*100

rk_reallygreen.sort_values('really_green_percent_country',ascending=False).head(10)

,country_long,capacity_mw_total,is_really_green,capacity_mw_really_green,really_green_percent_country
72,Palestine,7.60,1,7.60,100.00
90,Suriname,5.00,1,5.00,100.00
82,Saint Lucia,3.00,1,3.00,100.00
52,Kenya,"2,485.90",1,"1,145.20",46.07
102,Uruguay,"4,209.12",1,"1,606.96",38.18
78,Portugal,"14,744.86",1,"5,592.71",37.93
39,Honduras,"2,164.87",1,769.45,35.54
28,El Salvador,"1,418.50",1,474.00,33.42
100,United Kingdom,"97,155.28",1,"31,915.68",32.85
13,Burkina Faso,229.31,1,52.90,23.07


In [63]:
# Just checking...
df[df.country_long == 'Palestine']

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,generation_data_source,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017,is_green
18246,PSE,Palestine,Jericho,WKS0073950,7.60,31.93,35.46,Solar,NaN,NaN,NaN,NaN,NaN,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.55,15.45,14.67,16.29,15.01,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,1


## Making maps

### Point map `2 points`

Using the tool of your choice, make a point map of power plants in **one specific country**.

In [74]:
df[df.country == 'BRA'].to_csv('maps/brazil.csv')

**ANSWER**: Please check `maps/pointmap.png`

### Bubble map `2 points`

Make the point map above more exciting by coloring according to fuel type and sizing bubbles according to power production capacity.

**ANSWER**: Please check `maps/BubbleMap.png`

### Hex bin map `4 points`

In the country of your choice, make a hex bin map of hydroelectric power plants (or if there aren't any: something else!).

**ANSWER**: Please check `maps/Hexbin.png`